In [126]:
from __future__ import print_function
import random
import os
import sys
import copy
import math
import time
import pickle
import contextlib
import numpy as np
import pandas as pd
import multiprocessing as mp
from deap import tools
from deap import base, creator
from sklearn.cluster import KMeans
from multiprocessing import Pool
from six.moves.urllib.request import urlretrieve
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, Activation
from keras.optimizers import adam, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold, cross_val_score
%matplotlib inline

In [16]:
data_root = 'data'
german_data = os.path.join(data_root,'GermanCreditInput.xls')
german_label = os.path.join(data_root,'GermanCreditOutputClass1columnknn.xls')
australian_dataset = os.path.join(data_root,'australian dataset.xlsx')

In [101]:
class Tools :
    
    data_root = ''
    root_url = ''
    chromosomes = {}
    
    @staticmethod
    def download_progress_hook(count, blockSize, totalSize):
        """A hook to report the progress of a download. This is mostly intended for users with
        slow internet connections. Reports every 5% change in download progress."""
        global last_percent_reported
        percent = int(count * blockSize * 100 / totalSize)

        if last_percent_reported != percent:
            if percent % 5 == 0:
                sys.stdout.write("%s%%" % percent)
                sys.stdout.flush()
            else:
                sys.stdout.write(".")
                sys.stdout.flush()
            last_percent_reported = percent
    
    @staticmethod
    def read_df(filename, expected_bytes=None, force=False):
        """Download a file if not present, and make sure it's the right size."""
        dest_filename = os.path.join(Tools.data_root, filename)
        direc = dest_filename[:dest_filename.rfind('/')]
        if not os.path.exists(direc):
            os.makedirs(direc)
        if force or not os.path.exists(dest_filename):
            print('Attempting to download:', filename) 
            filename, _ = urlretrieve(Tools.root_url + filename, dest_filename, reporthook=Tools.download_progress_hook)
            print('\nDownload Complete!')        
        return np.array(pd.read_csv(filename, header=None))
    
    @staticmethod
    def keras_model(input_dim) :
        model = Sequential()
        model.add(Dense(200, input_dim=input_dim, kernel_initializer='glorot_normal', activation='selu'))
        model.add(Dense(2, kernel_initializer='normal', activation='softmax'))
        sgd = SGD(lr=.1)
        # loss could be "mse" too
        model.compile(loss='binary_crossentropy',metrics=['accuracy','binary_accuracy'],optimizer='rmsprop')
        return lambda: model
    
    @staticmethod
    def save_to_file(path,data) :
        with open(path + '.pkl', 'wb') as f:
            pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    @staticmethod    
    def load_from_file(path) :
        with open(path + '.pkl', 'rb') as f:
            return pickle.load(f)

In [127]:
x_data = np.array(pd.read_excel(german_data,header=None))
y_data = to_categorical(np.array(pd.read_excel(german_label,header=None)),2)
print('Dataset Shape : {}\nDataset Labels Shape : {}'.format(x_data.shape,y_data.shape))

Dataset Shape : (1000, 30)
Dataset Labels Shape : (1000, 2)


In [103]:
nn_config = {
    'train_cycles':600,
}

In [104]:
def fit(X,y,train_cycles) :
    model = Tools.keras_model(X.shape[1])
    classifier = KerasClassifier(build_fn=model, epochs=int(train_cycles),batch_size=32,verbose=1)
    return np.mean(cross_val_score(classifier, X, y, cv=10,verbose=1))

In [105]:
fit(x_data,y_data,nn_config['train_cycles'])

ValueError: Error when checking target: expected dense_56 to have shape (2,) but got array with shape (1,)

In [154]:
model = Sequential()
model.add(Dense(10, input_dim=30, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(5, kernel_initializer='glorot_normal', activation='selu'))
model.add(Dense(2, kernel_initializer='glorot_normal', activation='softmax'))
# loss could be "mse" too
opt = SGD(lr=0.000001)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='rmsprop')
model.fit(x_data,y_data, epochs=int(10000),batch_size=64,verbose=1,validation_split=.1)

Train on 900 samples, validate on 100 samples
Epoch 1/10000
900/900 [==============================] - 2s 2ms/step - loss: 4.6696 - acc: 0.6989 - val_loss: 5.0820 - val_acc: 0.6800
Epoch 2/10000
900/900 [==============================] - 0s 92us/step - loss: 4.6065 - acc: 0.6967 - val_loss: 4.8505 - val_acc: 0.5100
Epoch 3/10000
900/900 [==============================] - 0s 88us/step - loss: 3.6399 - acc: 0.6344 - val_loss: 2.6992 - val_acc: 0.6200
Epoch 4/10000
900/900 [==============================] - 0s 89us/step - loss: 3.1447 - acc: 0.6589 - val_loss: 4.6278 - val_acc: 0.6800
Epoch 5/10000
900/900 [==============================] - 0s 89us/step - loss: 3.3617 - acc: 0.6744 - val_loss: 2.3606 - val_acc: 0.7000
Epoch 6/10000
900/900 [==============================] - 0s 89us/step - loss: 3.1783 - acc: 0.6500 - val_loss: 4.8564 - val_acc: 0.6800
Epoch 7/10000
900/900 [==============================] - 0s 86us/step - loss: 3.2424 - acc: 0.6533 - val_loss: 2.1378 - val_acc: 0.7000
Epo

900/900 [==============================] - 0s 80us/step - loss: 0.7196 - acc: 0.6022 - val_loss: 0.6473 - val_acc: 0.6800
Epoch 61/10000
900/900 [==============================] - 0s 76us/step - loss: 0.7718 - acc: 0.5811 - val_loss: 0.7564 - val_acc: 0.5300
Epoch 62/10000
900/900 [==============================] - 0s 75us/step - loss: 0.7444 - acc: 0.5656 - val_loss: 0.8005 - val_acc: 0.4300
Epoch 63/10000
900/900 [==============================] - 0s 82us/step - loss: 0.6712 - acc: 0.6100 - val_loss: 0.7937 - val_acc: 0.4200
Epoch 64/10000
900/900 [==============================] - 0s 75us/step - loss: 0.7333 - acc: 0.5556 - val_loss: 0.6651 - val_acc: 0.6100
Epoch 65/10000
900/900 [==============================] - 0s 76us/step - loss: 0.6723 - acc: 0.5933 - val_loss: 0.6565 - val_acc: 0.6900
Epoch 66/10000
900/900 [==============================] - 0s 76us/step - loss: 0.6685 - acc: 0.5911 - val_loss: 0.7277 - val_acc: 0.4200
Epoch 67/10000
900/900 [==============================] 

Epoch 120/10000
900/900 [==============================] - 0s 73us/step - loss: 0.5607 - acc: 0.7022 - val_loss: 0.5470 - val_acc: 0.6800
Epoch 121/10000
900/900 [==============================] - 0s 80us/step - loss: 0.5587 - acc: 0.7044 - val_loss: 0.5754 - val_acc: 0.6800
Epoch 122/10000
900/900 [==============================] - 0s 73us/step - loss: 0.5690 - acc: 0.7033 - val_loss: 1.1365 - val_acc: 0.6800
Epoch 123/10000
900/900 [==============================] - 0s 76us/step - loss: 0.5835 - acc: 0.7022 - val_loss: 0.5503 - val_acc: 0.6800
Epoch 124/10000
900/900 [==============================] - 0s 109us/step - loss: 0.5544 - acc: 0.7033 - val_loss: 0.5526 - val_acc: 0.6800
Epoch 125/10000
900/900 [==============================] - 0s 79us/step - loss: 0.5669 - acc: 0.7033 - val_loss: 0.5372 - val_acc: 0.6800
Epoch 126/10000
900/900 [==============================] - 0s 80us/step - loss: 0.5682 - acc: 0.7033 - val_loss: 0.5420 - val_acc: 0.6800
Epoch 127/10000
900/900 [========

900/900 [==============================] - 0s 86us/step - loss: 0.5417 - acc: 0.7044 - val_loss: 0.6266 - val_acc: 0.5600
Epoch 180/10000
900/900 [==============================] - 0s 93us/step - loss: 0.5676 - acc: 0.6933 - val_loss: 0.6217 - val_acc: 0.6800
Epoch 181/10000
900/900 [==============================] - 0s 80us/step - loss: 0.5458 - acc: 0.7167 - val_loss: 1.6388 - val_acc: 0.3700
Epoch 182/10000
900/900 [==============================] - 0s 81us/step - loss: 0.5999 - acc: 0.6811 - val_loss: 0.5846 - val_acc: 0.6700
Epoch 183/10000
900/900 [==============================] - 0s 80us/step - loss: 0.5357 - acc: 0.7389 - val_loss: 0.5209 - val_acc: 0.6800
Epoch 184/10000
900/900 [==============================] - 0s 90us/step - loss: 0.5136 - acc: 0.7311 - val_loss: 0.6110 - val_acc: 0.6600
Epoch 185/10000
900/900 [==============================] - 0s 92us/step - loss: 0.5319 - acc: 0.7344 - val_loss: 0.5300 - val_acc: 0.6800
Epoch 186/10000
900/900 [=========================

900/900 [==============================] - 0s 86us/step - loss: 0.5272 - acc: 0.7544 - val_loss: 0.6277 - val_acc: 0.6100
Epoch 239/10000
900/900 [==============================] - 0s 85us/step - loss: 0.4971 - acc: 0.7400 - val_loss: 0.8037 - val_acc: 0.7000
Epoch 240/10000
900/900 [==============================] - 0s 84us/step - loss: 0.5089 - acc: 0.7489 - val_loss: 0.5204 - val_acc: 0.7000
Epoch 241/10000
900/900 [==============================] - 0s 84us/step - loss: 0.5119 - acc: 0.7367 - val_loss: 0.5360 - val_acc: 0.7400
Epoch 242/10000
900/900 [==============================] - 0s 90us/step - loss: 0.5292 - acc: 0.7478 - val_loss: 0.5121 - val_acc: 0.7000
Epoch 243/10000
900/900 [==============================] - 0s 81us/step - loss: 0.4826 - acc: 0.7433 - val_loss: 1.1400 - val_acc: 0.6900
Epoch 244/10000
900/900 [==============================] - 0s 83us/step - loss: 0.5324 - acc: 0.7244 - val_loss: 0.9476 - val_acc: 0.7000
Epoch 245/10000
900/900 [=========================

...

900/900 [==============================] - 0s 73us/step - loss: 0.3840 - acc: 0.8133 - val_loss: 0.5784 - val_acc: 0.7000
Epoch 5842/10000
900/900 [==============================] - 0s 79us/step - loss: 0.3897 - acc: 0.8111 - val_loss: 0.5802 - val_acc: 0.7500
Epoch 5843/10000
900/900 [==============================] - 0s 79us/step - loss: 0.3773 - acc: 0.8133 - val_loss: 0.5733 - val_acc: 0.6500
Epoch 5844/10000
900/900 [==============================] - 0s 76us/step - loss: 0.3886 - acc: 0.8211 - val_loss: 0.6143 - val_acc: 0.7400
Epoch 5845/10000
900/900 [==============================] - 0s 80us/step - loss: 0.3909 - acc: 0.8144 - val_loss: 0.6267 - val_acc: 0.6900
Epoch 5846/10000
900/900 [==============================] - 0s 80us/step - loss: 0.3852 - acc: 0.8156 - val_loss: 0.6130 - val_acc: 0.7200
Epoch 5847/10000
900/900 [==============================] - 0s 79us/step - loss: 0.3791 - acc: 0.8156 - val_loss: 0.8977 - val_acc: 0.7200
Epoch 5848/10000
900/900 [==================

900/900 [==============================] - 0s 78us/step - loss: 0.3806 - acc: 0.8189 - val_loss: 0.6235 - val_acc: 0.7600
Epoch 5901/10000
900/900 [==============================] - 0s 78us/step - loss: 0.3889 - acc: 0.8067 - val_loss: 1.0232 - val_acc: 0.7200
Epoch 5902/10000
900/900 [==============================] - 0s 84us/step - loss: 0.3939 - acc: 0.8156 - val_loss: 0.6948 - val_acc: 0.7600
Epoch 5903/10000
900/900 [==============================] - 0s 81us/step - loss: 0.3707 - acc: 0.8200 - val_loss: 0.5816 - val_acc: 0.7200
Epoch 5904/10000
900/900 [==============================] - 0s 80us/step - loss: 0.3833 - acc: 0.8089 - val_loss: 0.5778 - val_acc: 0.6800
Epoch 5905/10000
900/900 [==============================] - 0s 76us/step - loss: 0.3985 - acc: 0.8011 - val_loss: 0.6469 - val_acc: 0.7600
Epoch 5906/10000
900/900 [==============================] - 0s 78us/step - loss: 0.3781 - acc: 0.8356 - val_loss: 0.8344 - val_acc: 0.7300
Epoch 5907/10000
900/900 [==================

900/900 [==============================] - 0s 81us/step - loss: 0.3806 - acc: 0.8244 - val_loss: 0.7190 - val_acc: 0.7600
Epoch 5960/10000
900/900 [==============================] - 0s 78us/step - loss: 0.3774 - acc: 0.8167 - val_loss: 1.3963 - val_acc: 0.6500
Epoch 5961/10000
900/900 [==============================] - 0s 76us/step - loss: 0.4072 - acc: 0.8122 - val_loss: 0.7185 - val_acc: 0.7400
Epoch 5962/10000
900/900 [==============================] - 0s 76us/step - loss: 0.3872 - acc: 0.8211 - val_loss: 0.6878 - val_acc: 0.7300
Epoch 5963/10000
900/900 [==============================] - 0s 76us/step - loss: 0.3811 - acc: 0.8211 - val_loss: 1.0772 - val_acc: 0.6700
Epoch 5964/10000
900/900 [==============================] - 0s 81us/step - loss: 0.4015 - acc: 0.8122 - val_loss: 0.7248 - val_acc: 0.6800
Epoch 5965/10000
900/900 [==============================] - 0s 78us/step - loss: 0.3890 - acc: 0.8133 - val_loss: 0.6798 - val_acc: 0.7200
Epoch 5966/10000
900/900 [==================

900/900 [==============================] - 0s 80us/step - loss: 0.3748 - acc: 0.8111 - val_loss: 0.6020 - val_acc: 0.7400
Epoch 6019/10000
900/900 [==============================] - 0s 88us/step - loss: 0.3855 - acc: 0.8122 - val_loss: 0.7525 - val_acc: 0.7000
Epoch 6020/10000
900/900 [==============================] - 0s 80us/step - loss: 0.3766 - acc: 0.8200 - val_loss: 0.6065 - val_acc: 0.7400
Epoch 6021/10000
900/900 [==============================] - 0s 80us/step - loss: 0.3858 - acc: 0.8122 - val_loss: 0.5808 - val_acc: 0.7500
Epoch 6022/10000
900/900 [==============================] - 0s 76us/step - loss: 0.3729 - acc: 0.8189 - val_loss: 0.6501 - val_acc: 0.7600
Epoch 6023/10000
900/900 [==============================] - 0s 80us/step - loss: 0.4032 - acc: 0.8089 - val_loss: 0.8996 - val_acc: 0.7300
Epoch 6024/10000
900/900 [==============================] - 0s 82us/step - loss: 0.3810 - acc: 0.8244 - val_loss: 0.6863 - val_acc: 0.7300
Epoch 6025/10000
900/900 [==================

KeyboardInterrupt: 

In [157]:
np.sum(np.isnan(y_data))

0